In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
4024,2024-02-21,China Wcba Feminina,08:30,Zhejiang Golden Bulls F,Shanghai F,2.24,1.60,152.5,1.82,1.88,-3.5,2.07,1.89,8lvdejjg,0.446429,0.625000,0.549451,0.531915,0.071429,0.2,0.4,NaN,NaN,550.504,930.196350,1.689718,460.746,718.824017,1.560131,242.148,467.362,89.61,1740.00,0.0,1.0,0.0,0.0,0.235702,0.022933,0.064282,10.10,2.020,0.613861,0.000000,0.0,0.000000,19.13,3.826,0.156822,0.000000,0.0,0.000000
4025,2024-02-21,Eua Ncaa,22:00,Illinois State,Northern Iowa,1.91,1.83,131.5,1.83,1.83,131.5,1.83,1.83,Ecm1zX0b,0.523560,0.546448,0.546448,0.546448,0.070008,0.2,0.4,NaN,NaN,339.236,419.344373,1.236143,150.400,83.996031,0.558484,137.500,179.538,86.80,206.31,0.0,0.0,0.0,0.0,0.030251,0.000000,0.000000,-3.20,-0.640,-1.421875,0.601136,0.4,-0.201136,1.05,0.210,3.952381,0.431468,0.4,-0.031468
4026,2024-02-21,Eua Ncaa,21:00,Georgia State,Coastal Carolina,1.21,4.67,153.5,1.80,1.95,-10.5,2.00,11.00,ITWRob73,0.826446,0.214133,0.555556,0.512821,0.040579,0.6,0.8,NaN,NaN,220.516,131.711740,0.597289,366.910,297.160591,0.809900,138.308,443.890,102.20,215.25,0.0,0.0,0.0,0.0,0.832173,0.056569,0.979071,-0.87,-0.174,-1.206897,0.000000,0.0,0.000000,-5.00,-1.000,-3.670000,0.000000,0.0,0.000000
4027,2024-02-21,Eua Ncaa,21:00,Mercer,Citadel,1.50,2.69,133.5,1.80,1.95,-6.5,2.05,4.50,xGpvn8tL,0.666667,0.371747,0.555556,0.512821,0.038414,0.6,0.4,NaN,NaN,189.532,117.640083,0.620687,198.854,100.690058,0.506352,173.132,226.310,388.96,197.12,1.0,0.0,0.0,0.0,0.401650,0.056569,0.528981,2.85,0.570,0.877193,0.524406,0.6,0.075594,-5.00,-1.000,-1.690000,0.332945,0.2,-0.132945
4028,2024-02-21,Eua Ncaa,22:00,Drake,Belmont,1.20,4.80,156.5,1.91,1.91,-10.5,2.00,11.00,0fqcyDoh,0.833333,0.208333,0.523560,0.523560,0.041667,0.6,0.6,NaN,NaN,134.854,50.017382,0.370900,141.710,35.753015,0.252297,111.522,249.438,116.85,131.52,0.0,0.0,0.0,0.0,0.848528,0.000000,0.979071,1.35,0.270,0.740741,0.828431,1.0,0.171569,-3.63,-0.726,-5.234160,0.390471,0.4,0.009529
4029,2024-02-21,Eua Ncaa,22:00,Southern Illinois,Murray State,1.49,2.74,135.5,1.91,1.91,-6.5,2.05,4.50,zDyAYAVA,0.671141,0.364964,0.523560,0.523560,0.036104,0.8,0.2,NaN,NaN,108.548,26.960833,0.248377,174.214,80.520700,0.462194,108.384,222.136,89.70,285.84,0.0,0.0,0.0,0.0,0.417912,0.000000,0.528981,-2.64,-0.528,-0.928030,0.743841,0.6,-0.143841,0.19,0.038,45.789474,0.387108,0.4,0.012892
4030,2024-02-21,Europa Liga Waba Feminina,14:00,Partizan F,Sloga Pozega F,1.64,2.17,142.5,1.84,1.86,-3.5,1.96,2.35,CjW2HekD,0.609756,0.460829,0.543478,0.537634,0.070586,0.8,0.0,NaN,NaN,423.406,699.630709,1.652387,263.118,267.051747,1.014951,251.694,0.000,99.68,353.60,0.0,0.0,0.0,0.0,0.196728,0.007644,0.127968,-1.57,-0.314,-2.038217,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,08:30,China Wcba Feminina,Zhejiang Golden Bulls F,Shanghai F,152.5,1.82,0.9806,Over
1,21:00,Eua Ncaa,Georgia State,Coastal Carolina,153.5,1.80,1.0000,Over
2,21:00,Eua Ncaa,Mercer,Citadel,133.5,1.80,1.0000,Over
3,22:00,Eua Ncaa,Drake,Belmont,156.5,1.91,1.0000,Over
4,22:00,Eua Ncaa,Southern Illinois,Murray State,135.5,1.91,1.0000,Over
5,14:00,Europa Liga Waba Feminina,Partizan F,Sloga Pozega F,142.5,1.84,1.0000,Over
